In [110]:
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from notion_client import Client

In [130]:
NOTION_TOKEN = "secret_XKLyoisWT09GFbcYNmDfvYNkpcFIfgtrXQzvulVSfnA"
NOTION_DATABASE_ID = "1043b7f8-2e0b-806e-9e31-dd541a1ed332"

notion = Client(auth=NOTION_TOKEN)

In [131]:
notion.databases.query(**{
	"database_id": NOTION_DATABASE_ID,
    "filter": {
        
    }
})

{'object': 'list',
 'results': [{'object': 'page',
   'id': '1043b7f8-2e0b-8118-bbd8-ff0491633c0b',
   'created_time': '2024-09-17T14:27:00.000Z',
   'last_edited_time': '2024-09-17T14:27:00.000Z',
   'created_by': {'object': 'user',
    'id': '20a4298c-ff5d-4b13-9c5c-5d1b1e2a9dce'},
   'last_edited_by': {'object': 'user',
    'id': '20a4298c-ff5d-4b13-9c5c-5d1b1e2a9dce'},
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': '1043b7f8-2e0b-806e-9e31-dd541a1ed332'},
   'archived': False,
   'in_trash': False,
   'properties': {'专栏 ID': {'id': '%3BU%7D%3D',
     'type': 'rich_text',
     'rich_text': [{'type': 'text',
       'text': {'content': 'xhsal', 'link': None},
       'annotations': {'bold': False,
        'italic': False,
        'strikethrough': False,
        'underline': False,
        'code': False,
        'color': 'default'},
       'plain_text': 'xhsal',
       'href': None}]},
    '介绍': {'id': '%3EzVp',
     'type': 'rich_text',
     

In [112]:
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

In [113]:
url = f"https://xiaobot.net/p/shengcaiyoushu"
driver = setup_driver()
driver.get(url)

In [114]:
driver.title

'生财有术项目精选'

In [115]:
paper_info_elem = driver.find_element(By.CSS_SELECTOR, ".paper-info")

In [116]:
reader_num_elem = paper_info_elem.find_element(By.CSS_SELECTOR, ".stats div.stat:nth-child(1) .num")
reader_num_elem.text

'28654'

In [117]:
article_num_elem = paper_info_elem.find_element(By.CSS_SELECTOR, ".stats div.stat:nth-child(2) .num")
article_num_elem.text

'71'

In [118]:
intro_elem = paper_info_elem.find_element(By.CSS_SELECTOR, "p.intro")
intro_elem.text

'66个赚到钱的人，分享他们已经跑通的项目。\n\n【重要】添加服务官鱼丸微信(yuwan831)，领取配套的10张项目步骤拆解图；\n\n【阅读建议】先看拆解图，定位自己擅长的领域，便于找到更适合自己的项目。\n\n更多权益和规则，请看置顶帖。'

In [119]:
category_elem = driver.find_element(By.CSS_SELECTOR, ".category")
category_elem.text

'介绍\n内容\n免费预览'

In [120]:
# 如果找不到 tags, 就点击 category_elem 下的第二个元素
if not driver.find_elements(By.CSS_SELECTOR, "span.tag"):
    category_elem.find_elements(By.CSS_SELECTOR, "div")[1].click()

In [121]:
# 由于 div.tags 元素的父级定义了 overflow: hidden; 这将会导致取到的 tags 列表虽然是全的，但后面有部分 tag 的内容是空的
# 将 div.tags 元素上设置 overflow 设置为 auto 可以解决这个问题
driver.execute_script("document.querySelector('.tags').style.overflow = 'auto';")

In [123]:
tag_elems = driver.find_elements(By.CSS_SELECTOR, "span.tag")
# 打印找到 n 个 tags
print('找到', len(tag_elems), '个 tags')

# 遍历并打印 tags, 并打印出序号
for i, tag_elem in enumerate(tag_elems):
    print(i, tag_elem.text)
    
# 遍历 tags，去掉第一个标签（全部），并将前缀 # 字符去掉，以及后面的空格与数字。
tags = [tag.text.replace("#", "").split(" ")[0] for tag in tag_elems[1:]]
tags

找到 11 个 tags
0 全部
1 #电商带货 13
2 #技能服务 10
3 #整理资料 8
4 #进阶玩法 7
5 #流量变现 7
6 #本地服务 7
7 #线下实体 4
8 #社群团购 4
9 #中介服务 4
10 #工具产品 2


['电商带货',
 '技能服务',
 '整理资料',
 '进阶玩法',
 '流量变现',
 '本地服务',
 '线下实体',
 '社群团购',
 '中介服务',
 '工具产品']

In [99]:
# 获取主理人信息
host_elem = driver.find_element(By.CSS_SELECTOR, ".post .name")
host_elem.text

'生财有术'

In [66]:
# 要想获取到佣金比例数据，需要点击 div.share-paper 元素，点击后会在底部弹出一个模态对话框（div.el-overlay），里面有推广佣金比例信信息，
share_paper_elem = driver.find_element(By.CSS_SELECTOR, ".share-paper")
# 先判断是否有 .el-overlay 这个元素，如果有就不点击；反之则点击。
if not driver.find_elements(By.CSS_SELECTOR, ".el-overlay"):
    share_paper_elem.click()

# 获取推广佣金比例信息
commission_rate_elem = driver.find_element(By.CSS_SELECTOR, ".el-overlay .referText p span.text-theme")
# 去掉百分号
commission_rate = commission_rate_elem.text.replace("%", "")
commission_rate

'50'

In [72]:
# 获取文章标题
title_elem = driver.find_element(By.CSS_SELECTOR, ".paper_title h1")
print(title_elem.text)

# 也可以直接通过页面 title 直接获取专栏标题，更简单。
driver.title

生财有术项目精选


'生财有术项目精选'

In [74]:
# 获得 icon 链接
icon_elem = driver.find_element(By.CSS_SELECTOR, "img.avatar")
icon_elem.get_attribute("src")

'https://static.xiaobot.net/user-avatar/2022-11-21/11873/d577f4b18114480dc6d867f753a15e4b.png'